<a href="https://colab.research.google.com/github/estevaoeller/A2T/blob/main/Audio2Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Instalação da biblioteca do Whisper
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q soundfile # Dependência para ler alguns formatos de áudio


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

print("Conectando ao Google Drive...")
drive.mount('/content/drive')
print("Google Drive conectado com sucesso!")

Conectando ao Google Drive...
Mounted at /content/drive
Google Drive conectado com sucesso!


In [ ]:
import whisper
import os
import time
import datetime # Para obter a data e hora
import shutil   # Para copiar arquivos


from google.colab import drive

# PASSO 1: CONFIGURAÇÕES


# Caminhos para as pastas de trabalho
PASTA_RAIZ = "/content/drive/MyDrive/Audio2Text"
PASTA_DE_ORIGEM = os.path.join(PASTA_RAIZ, "1_Source")
PASTA_DE_DESTINO_TXT = os.path.join(PASTA_RAIZ, "2_Result")
PASTA_DE_ARQUIVAMENTO = os.path.join(PASTA_RAIZ, "3_Processed") # Nova pasta para arquivar


# --- Escolha do Modelo ---
# Opções: 'tiny', 'base', 'small', 'medium', 'large'
# 'base' é rápido e bom.
# 'medium' é mais preciso e um ótimo equilíbrio para a GPU do Colab.
# 'large' é o mais preciso, mas também o mais lento.
MODEL_SIZE = "medium"


# Lista de extensões de áudio que o script deve procurar
EXTENSOES_VALIDAS = ('.mp3', '.wav', '.m4a', '.flac', '.ogg', '.3gp', '.mp4', '.opus', '.aac')


# PASSO 2: CARREGAR O MODELO WHISPER
print(f"Carregando o modelo Whisper ('{MODEL_SIZE}')... Isso pode levar um momento.")
model = whisper.load_model(MODEL_SIZE)
print("Modelo carregado.")




Carregando o modelo Whisper ('medium')... Isso pode levar um momento.


100%|█████████████████████████████████████| 1.42G/1.42G [00:31<00:00, 47.9MiB/s]


Modelo carregado.


In [ ]:
# @title
# ==============================================================================
# CÉLULA 2: PROCESSAMENTO E ARQUIVAMENTO
# Execute esta célula sempre que tiver novos arquivos na pasta '1_Source'.
# ==============================================================================
import time

print("🔎 Verificando arquivos na pasta de origem...")
arquivos_na_pasta = os.listdir(PASTA_DE_ORIGEM)

if not any(f.lower().endswith(EXTENSOES_VALIDAS) for f in arquivos_na_pasta):
    print("✅ Nenhum arquivo de áudio novo encontrado na pasta de origem. Tudo certo!")
else:
    print(f"▶️ Iniciando processamento de {len(arquivos_na_pasta)} item(ns)...")
    for nome_do_arquivo in arquivos_na_pasta:
        if nome_do_arquivo.lower().endswith(EXTENSOES_VALIDAS):
            caminho_audio_origem = os.path.join(PASTA_DE_ORIGEM, nome_do_arquivo)
            print(f"\n--- Processando: {nome_do_arquivo} ---")

            try:
                # 1. Transcrição
                start_time = time.time()
                result = model.transcribe(caminho_audio_origem, language="pt", fp16=False)
                end_time = time.time()
                print(f"Transcrição concluída em {end_time - start_time:.2f}s.")

                # 2. Salvamento do .txt
                nome_base_arquivo = os.path.splitext(nome_do_arquivo)[0]
                nome_arquivo_txt = f"transcricao_{nome_base_arquivo}.txt"
                caminho_txt_destino = os.path.join(PASTA_DE_DESTINO_TXT, nome_arquivo_txt)
                with open(caminho_txt_destino, "w", encoding="utf-8") as f:
                    f.write(result["text"])
                print(f"Resultado salvo em: {PASTA_DE_DESTINO_TXT}")

                # 3. Arquivamento
                timestamp = datetime.datetime.now().strftime("%y%m%d_%H%M%S") # Adicionado segundos para evitar conflito
                nome_pasta_arquivamento = f"{timestamp}_{nome_base_arquivo}"
                caminho_pasta_arquivamento = os.path.join(PASTA_DE_ARQUIVAMENTO, nome_pasta_arquivamento)
                os.makedirs(caminho_pasta_arquivamento, exist_ok=True)

                shutil.copy(caminho_audio_origem, caminho_pasta_arquivamento)
                shutil.copy(caminho_txt_destino, caminho_pasta_arquivamento)
                print(f"Arquivos copiados para: {PASTA_DE_ARQUIVAMENTO}")

                # 4. Limpeza (opcional, mas recomendado): remove o arquivo original para não ser processado de novo
                os.remove(caminho_audio_origem)
                print(f"Arquivo original '{nome_do_arquivo}' removido da pasta de origem.")

            except Exception as e:
                print(f"❌ Ocorreu um erro ao processar o arquivo {nome_do_arquivo}: {e}")
        else:
            print(f"- Ignorando: {nome_do_arquivo}")

    print("\n✅ Processo finalizado para todos os arquivos!")

🔎 Verificando arquivos na pasta de origem...
▶️ Iniciando processamento de 1 item(ns)...

--- Processando: scs 250723.mp3 ---
Transcrição concluída em 783.46s.
Resultado salvo em: /content/drive/MyDrive/Audio2Text/2_Result
Arquivos copiados para: /content/drive/MyDrive/Audio2Text/3_Processed
Arquivo original 'scs 250723.mp3' removido da pasta de origem.

✅ Processo finalizado para todos os arquivos!


In [ ]:
import os
import re
import sys
import shutil # Needed if you want to copy files around, but mostly for this cell, just for context

# Make sure these variables are defined in your initial setup cell and are globally accessible
# PASTA_RAIZ = "/content/drive/MyDrive/Audio2Text"
# PASTA_DE_ARQUIVAMENTO = os.path.join(PASTA_RAIZ, "3_Processed")

# Import the custom corrections module
if PASTA_RAIZ not in sys.path:
    sys.path.append(PASTA_RAIZ)

try:
    from correcoes_personalizadas import CORRECOES_PT_BR
    print("Dicionário de correções personalizadas carregado para correção adicional.")
except ImportError:
    print("AVISO: Arquivo 'correcoes_personalizadas.py' não encontrado ou com erro. Nenhuma correção personalizada será aplicada nesta célula.")
    CORRECOES_PT_BR = {}


# ==============================================================================
# CÉLULA 3 (NOVA): CORREÇÃO ADICIONAL / RE-CORREÇÃO DE ARQUIVOS ARQUIVADOS
# Execute esta célula para aplicar correções a arquivos TXT já arquivados.
# ==============================================================================

# --- CONFIGURAÇÃO PARA ESTA CÉLULA ESPECÍFICA ---
# Você precisa definir QUAL ARQUIVO VOCÊ QUER CORRIGIR NOVAMENTE.
# NOME_BASE_DO_AUDIO: O nome original do seu arquivo de áudio (SEM EXTENSÃO).
# Ex: Se o seu arquivo original era 'minha_reuniao.mp3', defina como 'minha_reuniao'.
NOME_BASE_DO_AUDIO = "coloque_aqui_o_nome_base_do_seu_audio" # <-- ***MUDE ISTO***

# Prefixo do arquivo que você quer usar como base para a correção.
# Opções comuns: "transcricao" (para o original transcrito) ou "corrigido" (para a versão já corrigida).
PREFIXO_ARQUIVO_DE_ENTRADA = "transcricao" # <-- ***MUDE ISTO*** (se for "corrigido", por exemplo)

# Prefixo para o NOVO arquivo corrigido que será gerado por esta célula.
# Ex: 'final_', 'corrigido_v2_', 'revisado_'.
NOVO_PREFIXO_ARQUIVO_SAIDA = "final_" # <-- ***MUDE ISTO***


print(f"Iniciando correção adicional para o arquivo com nome base: '{NOME_BASE_DO_AUDIO}'")

# 1. Localizar a pasta de arquivamento correspondente em 3_Processed
# Procuramos por uma pasta que contenha o NOME_BASE_DO_AUDIO no seu nome (devido ao timestamp)
caminho_pasta_arquivamento_do_audio = None
for pasta_arquivada in os.listdir(PASTA_DE_ARQUIVAMENTO):
    if NOME_BASE_DO_AUDIO in pasta_arquivada:
        caminho_pasta_arquivamento_do_audio = os.path.join(PASTA_DE_ARQUIVAMENTO, pasta_arquivada)
        break

if not caminho_pasta_arquivamento_do_audio:
    print(f"❌ Erro: Não foi possível encontrar a pasta de arquivamento para o áudio '{NOME_BASE_DO_AUDIO}' em '{PASTA_DE_ARQUIVAMENTO}'. Verifique o nome base.")
else:
    print(f"Pasta de arquivamento encontrada: {caminho_pasta_arquivamento_do_audio}")

    # 2. Localizar o arquivo TXT de entrada (transcricao_*.txt ou corrigido_*.txt) dentro desta pasta
    caminho_arquivo_txt_entrada = None
    for f_name in os.listdir(caminho_pasta_arquivamento_do_audio):
        if f_name.lower().startswith(f"{PREFIXO_ARQUIVO_DE_ENTRADA}_") and f_name.lower().endswith(".txt"):
            caminho_arquivo_txt_entrada = os.path.join(caminho_pasta_arquivamento_do_audio, f_name)
            break

    if not caminho_arquivo_txt_entrada:
        print(f"❌ Erro: Arquivo '{PREFIXO_ARQUIVO_DE_ENTRADA}_{NOME_BASE_DO_AUDIO}.txt' não encontrado na pasta '{os.path.basename(caminho_pasta_arquivamento_do_audio)}'. Verifique o prefixo ou se o arquivo existe.")
    else:
        print(f"Arquivo de entrada para correção encontrado: {os.path.basename(caminho_arquivo_txt_entrada)}")

        try:
            # 3. Ler o conteúdo do arquivo TXT de entrada
            with open(caminho_arquivo_txt_entrada, "r", encoding="utf-8") as f:
                texto_para_corrigir = f.read()
            print("Conteúdo do arquivo de entrada lido.")

            # 4. Aplicar as correções personalizadas
            texto_processado = texto_para_corrigir
            for erro, correcao in CORRECOES_PT_BR.items():
                # Usa regex para substituir apenas a palavra/frase inteira, ignorando maiúsculas/minúsculas
                # Isso é crucial para lidar com as correções contextuais (frases)
                texto_processado = re.sub(r'\b' + re.escape(erro) + r'\b', correcao, texto_processado, flags=re.IGNORECASE)
            print("Correções personalizadas aplicadas ao texto.")

            # 5. Salvar o NOVO arquivo TXT corrigido na mesma pasta de arquivamento
            nome_arquivo_txt_saida = f"{NOVO_PREFIXO_ARQUIVO_SAIDA}{NOME_BASE_DO_AUDIO}.txt"
            caminho_arquivo_txt_saida_final = os.path.join(caminho_pasta_arquivamento_do_audio, nome_arquivo_txt_saida)

            with open(caminho_arquivo_txt_saida_final, "w", encoding="utf-8") as f:
                f.write(texto_processado)
            print(f"Texto CORRIGIDO (versão adicional) salvo em: {caminho_arquivo_txt_saida_final}")

        except Exception as e:
            print(f"❌ Ocorreu um erro ao processar o arquivo para correção adicional: {e}")

    print("\n✅ Processo de Correção Adicional finalizado!")

Dicionário de correções personalizadas carregado para correção adicional.
Iniciando correção adicional para o arquivo com nome base: 'coloque_aqui_o_nome_base_do_seu_audio'
❌ Erro: Não foi possível encontrar a pasta de arquivamento para o áudio 'coloque_aqui_o_nome_base_do_seu_audio' em '/content/drive/MyDrive/Audio2Text/3_Processed'. Verifique o nome base.
